# Basics

## Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Get and extract dataset

In [2]:
!cp /content/drive/MyDrive/Ynov/deep_learning/data/stanford_dogo.rar /content/stanford_dogo.rar

In [3]:
!unrar x /content/stanford_dogo.rar /content/

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Extracting  /content/stanford_dogo/Rottweiler/n02106550_10706.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_10709.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_10714.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_10952.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_10966.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_11002.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_11097.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_11105.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_11176.jpg         75%  OK 
Extracting  /content/stanford_dogo/Rottweiler/n02106550_11203.jpg         75%  OK 
Ex

In [5]:
ls

augmented/  drive/  sample_data/  stanford_dogo/  stanford_dogo.rar


# Pre Treatment

## Imports and test GPU

In [105]:
import numpy as np
import os
import cv2
import json
from google.colab.patches import cv2_imshow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Dropout, Activation, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.applications.vgg16 import VGG16

In [7]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Train test split

In [62]:
!mkdir train
!mkdir test

In [66]:
data_path = "/content/stanford_dogo/"
train_path = "/content/train/"
test_path = "/content/test/"
for folder_name in os.listdir(data_path):
    if not os.path.exists(os.path.join(train_path, folder_name)):
        os.mkdir(os.path.join(train_path, folder_name))
    if not os.path.exists(os.path.join(test_path, folder_name)):
        os.mkdir(os.path.join(test_path, folder_name))
    dogs = os.listdir(os.path.join(data_path, folder_name))
    for file_name in dogs[:int(0.8*len(dogs))]:
        os.rename(os.path.join(data_path, folder_name, file_name), os.path.join(train_path, folder_name, file_name))
    for file_name in dogs[int(0.8*len(dogs)):]:
        os.rename(os.path.join(data_path, folder_name, file_name), os.path.join(test_path, folder_name, file_name))

## Datagen and data augmentation

In [142]:
datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='constant',
        cval=0,
        validation_split=0.2)

train_generator = datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

val_generator = datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation')

Found 13172 images belonging to 120 classes.
Found 3246 images belonging to 120 classes.


In [106]:
with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

In [108]:
!cp /content/class_indices.json /content/drive/MyDrive/Ynov/deep_learning/class_indices.json

## Visualize augmented data

In [ ]:
rm -r augmented

In [ ]:
mkdir augmented

In [ ]:
for file_name in val_generator.filenames:
    print(file_name)

In [ ]:
i = 0
for batch in datagen.flow_from_directory(
        "/content/stanford_dogo/",
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical',
        subset='validation',
        save_to_dir='/content/augmented/'):

    i += 1
    if i > 20: # save 20 images
        break  # otherwise the generator would loop indefinitely

Found 4072 images belonging to 120 classes.


In [ ]:
ls augmented/

_1037_8372329.png  _1806_4507403.png  _2676_5554397.png  _517_5553325.png
_1038_9691534.png  _2019_8436319.png  _2872_1596770.png  _550_2410081.png
_1046_27633.png    _2387_2237458.png  _3127_9797027.png  _699_4451598.png
_1070_7550395.png  _2440_5234734.png  _3263_7161742.png
_1197_6522583.png  _2507_6076217.png  _3697_1604763.png
_1419_4968554.png  _2594_6474055.png  _3836_7872474.png


In [ ]:
for filename in os.listdir("/content/augmented/"):
    print(filename)
    img = cv2.imread("/content/augmented/" + filename)
    cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

# Handcraft CNN

In [143]:
cnn = Sequential()

cnn.add(BatchNormalization(input_shape=(224, 224, 3)))
cnn.add(Conv2D(filters=16, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(filters=32, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(filters=64, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(filters=128, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())

cnn.add(Conv2D(filters=256, kernel_size=3, kernel_initializer='he_normal', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(BatchNormalization())

cnn.add(GlobalAveragePooling2D())

cnn.add(Dense(120, activation='softmax'))

cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 111, 111, 16)      64        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 54, 54, 32)       

In [144]:
cnn.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=[CategoricalAccuracy(), Precision(), Recall()])

In [145]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    #ReduceLROnPlateau(factor=0.2, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint('cnn_v5.h5', verbose=1, save_best_only=False, save_weights_only=False)
]

In [146]:
hist = cnn.fit(
        train_generator,
        epochs=100,
        validation_data=val_generator,
        callbacks=callbacks)

Epoch 1/100
412/412 [==============================] - 190s 460ms/step - loss: 4.7104 - categorical_accuracy: 0.0287 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 4.8358 - val_categorical_accuracy: 0.0351 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00

Epoch 00001: saving model to cnn_v5.h5
Epoch 2/100
412/412 [==============================] - 189s 458ms/step - loss: 4.3151 - categorical_accuracy: 0.0559 - precision_3: 0.5780 - recall_3: 2.1683e-04 - val_loss: 4.2150 - val_categorical_accuracy: 0.0758 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00

Epoch 00002: saving model to cnn_v5.h5
Epoch 3/100
412/412 [==============================] - 187s 454ms/step - loss: 4.0406 - categorical_accuracy: 0.0835 - precision_3: 0.3838 - recall_3: 2.7418e-04 - val_loss: 4.0634 - val_categorical_accuracy: 0.0964 - val_precision_3: 0.3182 - val_recall_3: 0.0022

Epoch 00003: saving model to cnn_v5.h5
Epoch 4/100
412/412 [==============================] - 186s 452ms

In [147]:
cp cnn_v5.h5 /content/drive/MyDrive/Ynov/deep_learning/cnn_v5.h5

## Test Handcraft CNN

In [148]:
ls

augmented/          cnn_v5.h5     stanford_dogo/     train/
class_indices.json  drive/        stanford_dogo.rar  vgg_v1.h5
cnn_v4.h5           sample_data/  test/              vgg_v2.h5


In [149]:
indice_to_class = {}
for dog, indice in val_generator.class_indices.items():
    indice_to_class[indice] = dog

In [150]:
dogs = {}
for dog in val_generator.class_indices.keys():
    dogs[dog] = 0
for filename in os.listdir(os.path.join(test_path, "Afghan_hound/")):
    img = cv2.imread(os.path.join(test_path, "Afghan_hound/", filename))
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
    pred = cnn.predict(np.array([img]))
    #print(pred)
    dogs[indice_to_class[np.argmax(pred)]] += 1

In [151]:
dogs

{'Afghan_hound': 0,
 'African_hunting_dog': 0,
 'Airedale': 0,
 'American_Staffordshire_terrier': 0,
 'Appenzeller': 0,
 'Australian_terrier': 0,
 'Bedlington_terrier': 0,
 'Bernese_mountain_dog': 0,
 'Blenheim_spaniel': 0,
 'Border_collie': 0,
 'Border_terrier': 0,
 'Boston_bull': 0,
 'Bouvier_des_Flandres': 0,
 'Brabancon_griffon': 0,
 'Brittany_spaniel': 0,
 'Cardigan': 0,
 'Chesapeake_Bay_retriever': 0,
 'Chihuahua': 0,
 'Dandie_Dinmont': 0,
 'Doberman': 0,
 'English_foxhound': 0,
 'English_setter': 0,
 'English_springer': 0,
 'EntleBucher': 0,
 'Eskimo_dog': 0,
 'French_bulldog': 0,
 'German_shepherd': 0,
 'German_short-haired_pointer': 0,
 'Gordon_setter': 0,
 'Great_Dane': 0,
 'Great_Pyrenees': 0,
 'Greater_Swiss_Mountain_dog': 0,
 'Ibizan_hound': 0,
 'Irish_setter': 0,
 'Irish_terrier': 0,
 'Irish_water_spaniel': 0,
 'Irish_wolfhound': 0,
 'Italian_greyhound': 0,
 'Japanese_spaniel': 0,
 'Kerry_blue_terrier': 0,
 'Labrador_retriever': 0,
 'Lakeland_terrier': 0,
 'Leonberg': 0,


In [152]:
dogs_accuracy = {}
count_total = 0
tp_total = 0
for folder_name in os.listdir(test_path):
    count = 0
    tp = 0
    for filename in os.listdir(os.path.join(test_path, folder_name)):
        try:
            img = cv2.imread(os.path.join(test_path, folder_name, filename))
            img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
            pred = cnn.predict(np.array([img]))
            count += 1
            count_total += 1
            if val_generator.class_indices[folder_name] == np.argmax(pred):
                tp += 1
                tp_total += 1
        except:
            print(f"error on {os.path.join(test_path, folder_name, filename)}")
    dogs_accuracy[folder_name] = tp / count
accuracy = tp_total / count_total

Accuracy sur le jeu de test :


*   Sans data augmentation : 0.017
*   Avec data augmentation : 0.004



In [153]:
accuracy

0.004084574723690533

In [154]:
dogs_accuracy

{'Afghan_hound': 0.0,
 'African_hunting_dog': 0.0,
 'Airedale': 0.0,
 'American_Staffordshire_terrier': 0.0,
 'Appenzeller': 0.0,
 'Australian_terrier': 0.0,
 'Bedlington_terrier': 0.0,
 'Bernese_mountain_dog': 0.0,
 'Blenheim_spaniel': 0.0,
 'Border_collie': 0.0,
 'Border_terrier': 0.0,
 'Boston_bull': 0.0,
 'Bouvier_des_Flandres': 0.0,
 'Brabancon_griffon': 0.0,
 'Brittany_spaniel': 0.0,
 'Cardigan': 0.0,
 'Chesapeake_Bay_retriever': 0.0,
 'Chihuahua': 0.0,
 'Dandie_Dinmont': 0.0,
 'Doberman': 0.0,
 'English_foxhound': 0.0,
 'English_setter': 0.0,
 'English_springer': 0.0,
 'EntleBucher': 0.0,
 'Eskimo_dog': 0.0,
 'French_bulldog': 0.0,
 'German_shepherd': 0.0,
 'German_short-haired_pointer': 0.0,
 'Gordon_setter': 0.0,
 'Great_Dane': 0.0,
 'Great_Pyrenees': 0.0,
 'Greater_Swiss_Mountain_dog': 0.0,
 'Ibizan_hound': 0.0,
 'Irish_setter': 0.0,
 'Irish_terrier': 0.0,
 'Irish_water_spaniel': 0.0,
 'Irish_wolfhound': 0.0,
 'Italian_greyhound': 0.0,
 'Japanese_spaniel': 0.02702702702702703

# Transfer Learning

In [155]:
vgg_org = VGG16()
vgg_org.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [156]:
vgg = Sequential()
for layer in vgg_org.layers[:-1]:
    layer.trainable = False
    vgg.add(layer)
vgg.add(Dense(120, activation='softmax'))

In [157]:
vgg.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [158]:
vgg.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=[CategoricalAccuracy(), Precision(), Recall()])

In [159]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    #ReduceLROnPlateau(factor=0.2, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint('vgg_v3.h5', verbose=1, save_best_only=False, save_weights_only=False)
]

In [160]:
hist = vgg.fit(
        train_generator,
        epochs=100,
        validation_data=val_generator,
        callbacks=callbacks)

Epoch 1/100
412/412 [==============================] - 206s 498ms/step - loss: 5.6526 - categorical_accuracy: 0.0188 - precision_4: 0.1338 - recall_4: 6.0709e-05 - val_loss: 5.0229 - val_categorical_accuracy: 0.0447 - val_precision_4: 0.3333 - val_recall_4: 3.0807e-04

Epoch 00001: saving model to vgg_v3.h5
Epoch 2/100
412/412 [==============================] - 205s 498ms/step - loss: 4.8281 - categorical_accuracy: 0.0630 - precision_4: 0.3415 - recall_4: 0.0013 - val_loss: 4.6282 - val_categorical_accuracy: 0.0730 - val_precision_4: 0.4138 - val_recall_4: 0.0037

Epoch 00002: saving model to vgg_v3.h5
Epoch 3/100
412/412 [==============================] - 208s 505ms/step - loss: 4.5600 - categorical_accuracy: 0.0825 - precision_4: 0.3860 - recall_4: 0.0053 - val_loss: 4.5752 - val_categorical_accuracy: 0.0718 - val_precision_4: 0.4815 - val_recall_4: 0.0080

Epoch 00003: saving model to vgg_v3.h5
Epoch 4/100
412/412 [==============================] - 206s 501ms/step - loss: 4.3834 - c

In [161]:
cp vgg_v3.h5 /content/drive/MyDrive/Ynov/deep_learning/vgg_v3.h5

## Test transfer learning

In [162]:
ls

augmented/          cnn_v5.h5     stanford_dogo/     train/     vgg_v3.h5
class_indices.json  drive/        stanford_dogo.rar  vgg_v1.h5
cnn_v4.h5           sample_data/  test/              vgg_v2.h5


In [163]:
indice_to_class = {}
for dog, indice in val_generator.class_indices.items():
    indice_to_class[indice] = dog

In [164]:
dogs = {}
for dog in val_generator.class_indices.keys():
    dogs[dog] = 0
for filename in os.listdir(os.path.join(test_path, "Afghan_hound/")):
    img = cv2.imread(os.path.join(test_path, "Afghan_hound/", filename))
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
    pred = vgg.predict(np.array([img]))
    dogs[indice_to_class[np.argmax(pred)]] += 1

In [165]:
dogs

{'Afghan_hound': 34,
 'African_hunting_dog': 0,
 'Airedale': 0,
 'American_Staffordshire_terrier': 0,
 'Appenzeller': 0,
 'Australian_terrier': 0,
 'Bedlington_terrier': 0,
 'Bernese_mountain_dog': 0,
 'Blenheim_spaniel': 0,
 'Border_collie': 0,
 'Border_terrier': 0,
 'Boston_bull': 0,
 'Bouvier_des_Flandres': 0,
 'Brabancon_griffon': 0,
 'Brittany_spaniel': 0,
 'Cardigan': 0,
 'Chesapeake_Bay_retriever': 0,
 'Chihuahua': 0,
 'Dandie_Dinmont': 0,
 'Doberman': 0,
 'English_foxhound': 0,
 'English_setter': 1,
 'English_springer': 0,
 'EntleBucher': 0,
 'Eskimo_dog': 0,
 'French_bulldog': 0,
 'German_shepherd': 0,
 'German_short-haired_pointer': 0,
 'Gordon_setter': 0,
 'Great_Dane': 0,
 'Great_Pyrenees': 0,
 'Greater_Swiss_Mountain_dog': 0,
 'Ibizan_hound': 0,
 'Irish_setter': 0,
 'Irish_terrier': 0,
 'Irish_water_spaniel': 0,
 'Irish_wolfhound': 0,
 'Italian_greyhound': 0,
 'Japanese_spaniel': 0,
 'Kerry_blue_terrier': 0,
 'Labrador_retriever': 0,
 'Lakeland_terrier': 0,
 'Leonberg': 0,

In [166]:
dogs_accuracy = {}
count_total = 0
tp_total = 0
for folder_name in os.listdir(test_path):
    count = 0
    tp = 0
    for filename in os.listdir(os.path.join(test_path, folder_name)):
        try:
            img = cv2.imread(os.path.join(test_path, folder_name, filename))
            img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
            pred = vgg.predict(np.array([img]))
            count += 1
            count_total += 1
            if val_generator.class_indices[folder_name] == np.argmax(pred):
                tp += 1
                tp_total += 1
        except:
            print(f"error on {os.path.join(test_path, folder_name, filename)}")
    dogs_accuracy[folder_name] = tp / count
accuracy = tp_total / count_total

Accuracy sur le jeu de test :


*   Sans data augmentation : 0.21
*   Avec data augmentation : 0.23

In [167]:
accuracy

0.22921672272945698

In [168]:
dogs_accuracy

{'Afghan_hound': 0.7083333333333334,
 'African_hunting_dog': 0.6470588235294118,
 'Airedale': 0.0,
 'American_Staffordshire_terrier': 0.12121212121212122,
 'Appenzeller': 0.0,
 'Australian_terrier': 0.175,
 'Bedlington_terrier': 0.32432432432432434,
 'Bernese_mountain_dog': 0.0,
 'Blenheim_spaniel': 0.2631578947368421,
 'Border_collie': 0.3,
 'Border_terrier': 0.0,
 'Boston_bull': 0.7297297297297297,
 'Bouvier_des_Flandres': 0.26666666666666666,
 'Brabancon_griffon': 0.03225806451612903,
 'Brittany_spaniel': 0.0,
 'Cardigan': 0.5806451612903226,
 'Chesapeake_Bay_retriever': 0.0,
 'Chihuahua': 0.12903225806451613,
 'Dandie_Dinmont': 0.3055555555555556,
 'Doberman': 0.0,
 'English_foxhound': 0.875,
 'English_setter': 0.42424242424242425,
 'English_springer': 0.03125,
 'EntleBucher': 0.024390243902439025,
 'Eskimo_dog': 0.03333333333333333,
 'French_bulldog': 0.3125,
 'German_shepherd': 0.0,
 'German_short-haired_pointer': 0.06451612903225806,
 'Gordon_setter': 0.03225806451612903,
 'Grea